In [ ]:
# !pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
import gym_super_mario_bros 
import os 

from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
SIMPLE_MOVEMENT

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
done = True 

for step in range(2):
    if done:
        env.reset() 
    state, reward, done, info = env.step(env.action_space.sample())
    env.render() 
    
env.close()

In [ ]:
env.close()

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state = env.reset()

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
plt.figure(figsize=(20, 16))
for idx in range(state.shape[3]):
    plt.subplot(1, 4, idx + 1)
    plt.imshow(state[0][:, :, idx])
plt.show()

In [ ]:
class TrainingAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainingAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq 
        self.save_path = save_path 
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [ ]:
CHECKPOINT_DIR = '../Tutorial/Models/Mario/train/'
LOG_DIR = '../Tutorial/Models/Mario/logs/'

In [ ]:
callback = TrainingAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [ ]:
model = PPO('CnnPolicy', env, verbose=1, learning_rate=0.000001, n_steps=512)

In [ ]:
model.learn(total_timesteps=1000000)

In [ ]:
model.save('Mario_model')

In [ ]:
state = env.reset()

while True: 
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()
    
env.close()

In [ ]:
env.close()